## Customer churn prediction using logistic regression

This project is to create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lreg').getOrCreate()

In [4]:
data = spark.read.csv('data/customer_churn.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [7]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
final_data = output.select('features','churn')

In [31]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [32]:
from pyspark.ml.classification import LogisticRegression

In [33]:
lr_churn = LogisticRegression(labelCol='churn')

In [34]:
fitted_churn_model = lr_churn.fit(train_churn)

In [35]:
training_sum = fitted_churn_model.summary

In [36]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                654|                654|
|   mean| 0.1819571865443425|0.13761467889908258|
| stddev|0.38610453876688716|0.34475878532656845|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [37]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [38]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [39]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,11128.95,1....|    0|[4.04276443397641...|[0.98275375987096...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.20647376633295...|[0.98531990349271...|       0.0|
|[29.0,13255.05,1....|    0|[4.07372092221227...|[0.98327066926601...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.76738593453599...|[0.99688182996411...|       0.0|
|[30.0,8677.28,1.0...|    0|[3.96404073515634...|[0.98136752003667...|       0.0|
|[30.0,8874.83,0.0...|    0|[2.92176031865408...|[0.94891170377450...|       0.0|
|[30.0,10183.98,1....|    0|[2.73859636704974...|[0.93926607535225...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.10089082264150...|[0.95692947566726...|       0.0|
|[31.0,5387.75,0.0...|    0|[2.22865943802287...|[0.90279377859310...|       0.0|
|[31.0,7073.61,0

In [40]:
#Evaluvating using AUC
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [41]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [42]:
auc

0.7717179294823705

In [50]:
#Evaluating new customer data with this model
final_lr_model = lr_churn.fit(final_data)

In [51]:
new_customers = spark.read.csv('data/new_customers.csv',inferSchema=True, header=True)

In [52]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [53]:
test_new_customers = assembler.transform(new_customers)

In [54]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [55]:
final_results = final_lr_model.transform(test_new_customers)

In [56]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Per the model, we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!